In [2]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import json
import requests
import time

Retrieving and Filtering Dataset

In [20]:
url = 'https://data.nsw.gov.au/data/dataset/aefcde60-3b0c-4bc0-9af1-6fe652944ec2/resource/21304414-1ff1-4243-a5d2-f52778048b29/download/confirmed_cases_table1_location.csv'  

df = pd.read_csv(url)

df = df[['notification_date', 'postcode', 'lga_name19']]
df

,notification_date,postcode,lga_name19
0,2020-01-25,2134,Burwood (A)
1,2020-01-25,2121,Parramatta (C)
2,2020-01-25,2071,Ku-ring-gai (A)
3,2020-01-27,2033,Randwick (C)
4,2020-03-01,2163,Fairfield (C)
...,...,...,...
6522,2021-07-15,2164,Fairfield (C)
6523,2021-07-15,2176,Fairfield (C)
6524,2021-07-15,2177,Fairfield (C)
6525,2021-07-15,2165,Fairfield (C)


In [21]:
for i, row in df.iterrows():
  df.at[i, 'notification_date'] = datetime.strptime(df.at[i, 'notification_date'], '%Y-%m-%d')

df

,notification_date,postcode,lga_name19
0,2020-01-25 00:00:00,2134,Burwood (A)
1,2020-01-25 00:00:00,2121,Parramatta (C)
2,2020-01-25 00:00:00,2071,Ku-ring-gai (A)
3,2020-01-27 00:00:00,2033,Randwick (C)
4,2020-03-01 00:00:00,2163,Fairfield (C)
...,...,...,...
6522,2021-07-15 00:00:00,2164,Fairfield (C)
6523,2021-07-15 00:00:00,2176,Fairfield (C)
6524,2021-07-15 00:00:00,2177,Fairfield (C)
6525,2021-07-15 00:00:00,2165,Fairfield (C)


In [22]:
current_date = datetime.now()
N_days_ago = 14 #CHANGE THIS VARIABLE TO CHANGE THE TIMEFRAME

filter_date = current_date - timedelta(days=N_days_ago)

working_data = df[df.notification_date > filter_date]

working_data

,notification_date,postcode,lga_name19
5741,2021-07-03 00:00:00,2167,Campbelltown (C) (NSW)
5742,2021-07-03 00:00:00,2167,Campbelltown (C) (NSW)
5743,2021-07-03 00:00:00,2156,The Hills Shire (A)
5744,2021-07-03 00:00:00,2190,Canterbury-Bankstown (A)
5745,2021-07-03 00:00:00,2222,Georges River (A)
...,...,...,...
6522,2021-07-15 00:00:00,2164,Fairfield (C)
6523,2021-07-15 00:00:00,2176,Fairfield (C)
6524,2021-07-15 00:00:00,2177,Fairfield (C)
6525,2021-07-15 00:00:00,2165,Fairfield (C)


In [23]:
working_data.drop_duplicates(subset=['postcode'], inplace=True)
working_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,notification_date,postcode,lga_name19
5741,2021-07-03 00:00:00,2167,Campbelltown (C) (NSW)
5743,2021-07-03 00:00:00,2156,The Hills Shire (A)
5744,2021-07-03 00:00:00,2190,Canterbury-Bankstown (A)
5745,2021-07-03 00:00:00,2222,Georges River (A)
5746,2021-07-03 00:00:00,2030,Waverley (A)
...,...,...,...
6433,2021-07-14 00:00:00,2136,Strathfield (A)
6447,2021-07-14 00:00:00,2233,Sutherland Shire (A)
6455,2021-07-15 00:00:00,2768,Blacktown (C)
6465,2021-07-15 00:00:00,2570,Camden (A)


Determining Longitude and Latitude




In [24]:
working_data.reset_index(drop=True, inplace=True)
working_data

working_data['latitude'] = None
working_data['longitude'] = None

working_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,notification_date,postcode,lga_name19,latitude,longitude
0,2021-07-03 00:00:00,2167,Campbelltown (C) (NSW),None,None
1,2021-07-03 00:00:00,2156,The Hills Shire (A),None,None
2,2021-07-03 00:00:00,2190,Canterbury-Bankstown (A),None,None
3,2021-07-03 00:00:00,2222,Georges River (A),None,None
4,2021-07-03 00:00:00,2030,Waverley (A),None,None
...,...,...,...,...,...
103,2021-07-14 00:00:00,2136,Strathfield (A),None,None
104,2021-07-14 00:00:00,2233,Sutherland Shire (A),None,None
105,2021-07-15 00:00:00,2768,Blacktown (C),None,None
106,2021-07-15 00:00:00,2570,Camden (A),None,None


In [25]:
base_url = 'https://nominatim.openstreetmap.org/search'

for i, row in working_data.iterrows():
  location = str(working_data.at[i, 'postcode']) 
  my_params= {'q': location + ', NSW, Australia','format':'json','limit':'1'}
  response = requests.get(base_url, params = my_params)
  match = response.json()

  longitude = float(match[0]['lon'])
  latitude = float(match[0]['lat'])

  working_data.at[i, 'longitude'] = longitude
  working_data.at[i, 'latitude'] = latitude

  time.sleep(1)



working_data

2167
2156
2190
2222
2030
2154
2210
2165
2135
2176
2153
nan
2144
2063
2026
2200
2020
2036
2032
2142
2046
2042
2567
2079
2035
2205
2164
2171
2033
2150
2197
2022
2025
2206
2127
2221
2162
2234
2134
2145
2170
2213
2209
2031
2224
2192
2118
2218
2177
2194
2141
2034
2766
2198
2216
2219
2763
2217
2163
2759
2166
2229
2143
2196
2131
2161
2220
2021
2756
2223
2263
2040
2000
2160
2191
2009
2008
2146
2195
2750
2179
2199
2093
2148
2428
2168
2023
2112
2193
2207
2018
2085
2228
2232
2765
2752
2225
2138
2747
2767
2212
2155
2147
2136
2233
2768
2570
2565


,notification_date,postcode,lga_name19,latitude,longitude
0,2021-07-03 00:00:00,2167,Campbelltown (C) (NSW),-33.9695,150.889
1,2021-07-03 00:00:00,2156,The Hills Shire (A),-33.6832,150.985
2,2021-07-03 00:00:00,2190,Canterbury-Bankstown (A),-33.901,151.055
3,2021-07-03 00:00:00,2222,Georges River (A),-33.9662,151.09
4,2021-07-03 00:00:00,2030,Waverley (A),-33.8642,151.278
...,...,...,...,...,...
103,2021-07-14 00:00:00,2136,Strathfield (A),-33.8894,151.085
104,2021-07-14 00:00:00,2233,Sutherland Shire (A),-34.0677,151.012
105,2021-07-15 00:00:00,2768,Blacktown (C),-33.7302,150.931
106,2021-07-15 00:00:00,2570,Camden (A),-34.0127,150.718


In [26]:
desired_suburb = 'Mosman' #CHANGE THIS TO CHANGE THE SUBURB

base_url = 'https://nominatim.openstreetmap.org/search'

my_params= {'q': desired_suburb + ', NSW, Australia','format':'json','limit':'1'}
response = requests.get(base_url, params = my_params)
match = response.json()

desired_longitude = float(match[0]['lon'])
desired_latitude = float(match[0]['lat'])


In [27]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(desired_latitude, desired_longitude, latitude, longitude):
# approximate radius of earth in km
  R = 6373.0

  lat1 = radians(latitude)
  lon1 = radians(longitude)
  lat2 = radians(desired_latitude)
  lon2 = radians(desired_longitude)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c

  return distance

In [28]:
for i, row in working_data.iterrows():
  distance = calculate_distance(desired_latitude, desired_longitude, working_data.at[i, 'latitude'], working_data.at[i, 'longitude'])

  if distance > 10: #CHANGE THIS TO CHANGE THE RADIUS
    working_data.drop(i, inplace=True)

working_data

36.326343881654026
29.07718647366626
19.18460732335173
20.816491650597662
4.893074001250563
24.04629270065819
21.87055468645798
27.892166921232402
15.34187679080085
32.874242600068136
24.324154247474038
5.839445750197055
20.311041579415487
3.1891309676682145
7.027364569196108
22.16730989180291
11.881637954380595
15.459225249569217
10.548460554021677
21.420382098199457
11.31577204753313
9.601449382723791
51.89086650143419
21.278690951906295
12.446970834519318
14.675715913071118
30.9291373066748
37.37492470083053
9.31548735088972
21.848793505466244
24.304626508115952
6.905624255452591
6.291585312856561
14.552092206941133
16.0052239512057
21.335641790925408
23.0164637740428
28.67666061903066
13.973719587881561
26.924239396639457
31.79185147407551
26.697466667126136
20.95210442997822
9.518835223347233
24.611776248287505
16.96412513437543
20.229757335420995
19.14992295695106
34.05594707505748
15.898425005637257
18.85769064282378
9.349288494710931
37.388663901697086
25.395503290750938
16.268

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,notification_date,postcode,lga_name19,latitude,longitude
4,2021-07-03 00:00:00,2030,Waverley (A),-33.8642,151.278
11,2021-07-03 00:00:00,NaN,NaN,-33.8738,151.21
13,2021-07-04 00:00:00,2063,Willoughby (C),-33.8105,151.22
14,2021-07-04 00:00:00,2026,Waverley (A),-33.8902,151.268
21,2021-07-04 00:00:00,2042,Sydney (C),-33.8971,151.179
28,2021-07-05 00:00:00,2033,Randwick (C),-33.9117,151.222
31,2021-07-05 00:00:00,2022,Waverley (A),-33.892,151.25
32,2021-07-05 00:00:00,2025,Woollahra (A),-33.8867,151.243
43,2021-07-06 00:00:00,2031,Randwick (C),-33.9157,151.245
51,2021-07-07 00:00:00,2034,Randwick (C),-33.9141,151.249


In [29]:
working_data.reset_index(drop=True, inplace=True)
working_data

,notification_date,postcode,lga_name19,latitude,longitude
0,2021-07-03 00:00:00,2030,Waverley (A),-33.8642,151.278
1,2021-07-03 00:00:00,NaN,NaN,-33.8738,151.21
2,2021-07-04 00:00:00,2063,Willoughby (C),-33.8105,151.22
3,2021-07-04 00:00:00,2026,Waverley (A),-33.8902,151.268
4,2021-07-04 00:00:00,2042,Sydney (C),-33.8971,151.179
5,2021-07-05 00:00:00,2033,Randwick (C),-33.9117,151.222
6,2021-07-05 00:00:00,2022,Waverley (A),-33.892,151.25
7,2021-07-05 00:00:00,2025,Woollahra (A),-33.8867,151.243
8,2021-07-06 00:00:00,2031,Randwick (C),-33.9157,151.245
9,2021-07-07 00:00:00,2034,Randwick (C),-33.9141,151.249
